In [4]:
import re
import requests

from tradeskill_reader import Item, Recipe, Misc
from tqdm.notebook import tqdm

In [2]:
profession_collection = "https://www.wowhead.com/classic/spells/secondary-skills/cooking#0+17+18+1"
# profession_collection = "https://www.wowhead.com/wotlk/spells/secondary-skills/cooking#0+17"

req = requests.get(profession_collection, 'html.parser')

cookbook = re.findall("var listviewspells = \[(.*?)\];", req.text)[0]
spells = re.findall("\{(.*?)\}", cookbook)

# Save the file for later use.
# with open("wowhead_archive/classic_cooking.html", 'w') as f:
#     f.write(req.text)
#     f.close()

### Find all items and their names

We only use this to comment their names in the code to improve readability.

In [20]:
items_on_wowhead = re.findall("WH\.Gatherer\.addData\(3, 4, \{(.*?)\);", req.text)[0]
items_on_wowhead = re.findall('"(.*?)flags2":\d+\},', items_on_wowhead)

In [21]:
items = [Item.read(item) for item in items_on_wowhead]
id_dict = {item.id: item.name for item in items}

We finally go through each recipe and create a LUA dictionary that the addon will read from.

In [6]:
with open("CookingRecipes.lua", 'w') as f:
    f.write("local recipes = {\n")
    
    for i, spell in enumerate(spells):
        spell = Recipe.read(spell)
        
        # Actions and profession upgrades are also saved here.
        if type(spell) is Recipe:
            f.write(spell.get_LUA_list(id_dict))
    
    f.write("}\n\n")
    f.write("BitesCookBook_Recipes = recipes")
    f.close()

Find mobs that drop the item.

In [23]:
from json import loads

item_page = f"https://www.wowhead.com/classic/item={2681}"
req = requests.get(item_page, 'html.parser')

# with open("wowhead_archive/item_test.html", 'w') as f:
#     f.write(req.text)
#     f.close()

In [41]:
def find_host_mobs(item_id: int):
    item_page = f"https://www.wowhead.com/classic/item={item_id}"
    req = requests.get(item_page, 'html.parser')

    try:
        items_on_wowhad = re.findall("dropped-by(.*?)\n\}\);", req.text, re.DOTALL)[0]
    except IndexError:
        return
    
    # Only keep data: onwards
    data = re.findall("data:(.+)", items_on_wowhad)[0]
    valid_enemies = {}
    for enemy in loads(data[:-1]):
        if enemy["outof"] < 100:
            continue
        
        drop_chance = enemy["count"]/enemy["outof"]
        mob_id = enemy["id"]
        if drop_chance < 0.1:
            continue
        
        # Add the mob to the list of valid enemies as a dictionary item.
        valid_enemies[mob_id] = float("{:.2f}".format(drop_chance))
    
    return valid_enemies

In [25]:
for item_id in tqdm(id_dict):
    result = find_host_mobs(item_id)
    if result is None:
        continue
    else:
        print(result)
        break

  0%|          | 0/170 [00:00<?, ?it/s]

{1132: '0.31', 10356: '0.34', 12431: '0.40', 3056: '0.33', 1133: '0.67', 1549: '0.79', 1923: '0.58', 1765: '0.80', 1138: '0.62', 1131: '0.69', 1922: '0.68', 2958: '0.59', 1547: '0.59', 2959: '0.67', 525: '0.62', 2960: '0.78', 1548: '0.68', 1766: '0.78', 118: '0.80'}


In [54]:
with open("wowhead_archive/item_test.html", 'w') as f:
    for item_id in tqdm(id_dict):
        result = find_host_mobs(item_id)
        if result is None:
            continue
        else:
            f.write(str(result))
            f.write("\n")
            # break
    f.close()

  0%|          | 0/170 [00:00<?, ?it/s]

# An alternative way

In [43]:
# For each recipe (aka spell) find its ingredient and add it to a dictionary as a key.
# The value of the dictionary should be another dictionary with the a key named "Name" that has value of the item name.
# Another key should be called "DroppedBy" and have a value of a dictionary with the mob id as the key and the drop chance as the value.
recipes = [Recipe.read(spell) for spell in spells]
ingredient_dict = {}

for recipe in tqdm(recipes):
    if type(recipe) is not Recipe:
        continue
    
    for ingredient_id, ammount in recipe.reagents:
        if ingredient_id not in ingredient_dict:
            ingredient_dict[ingredient_id] = {"Name": id_dict[ingredient_id], "DroppedBy": find_host_mobs(ingredient_id)}
    
    break

  0%|          | 0/86 [00:00<?, ?it/s]